In [2]:
# Importing basic libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import string
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Importing libraries for sentiment analysis LSTM model

import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding,LSTM,Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model


In [3]:
# Reading the dataset

train = pd.read_csv('airline_df_nlped.csv')

In [4]:
train.head()

,Country,Airline,Review,Sentiment,Cleaned_Review
0,China,Air China,los angeles to beijing return. food low qualit...,Negative,los angeles to beijing return . food low quali...
1,China,Air China,round to trip from hong kong to munich. the ma...,Negative,round to trip from hong kong to munich . the m...
2,China,Air China,sydney to beijing to paris then rome to beijin...,Negative,sydney to beijing to paris then rome to beijin...
3,China,Air China,london to sydney return via beijing. a cheap f...,Negative,london to sydney return via beijing . a cheap ...
4,China,Air China,beijing to shanghai. only one check to in desk...,Positive,beijing to shanghai . only one check to in des...


In [5]:
train.shape

(15206, 5)

In [6]:
train.isna().sum()

Country           0
Airline           0
Review            0
Sentiment         0
Cleaned_Review    0
dtype: int64

In [13]:
# Model Building

# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

X = train['Review']

y = train['Sentiment']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [15]:
# Tokenizing the dataset

tokenizer = Tokenizer(num_words=5000,split=' ')

tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)

X_test = tokenizer.texts_to_sequences(X_test)

# Padding the dataset

from keras.utils import pad_sequences

X_train = pad_sequences(X_train,padding='post',maxlen=100)

X_test = pad_sequences(X_test,padding='post',maxlen=100)

# Building the LSTM model

model = Sequential()

model.add(Embedding(5000,128,input_length=X_train.shape[1]))

# A bidirectional LSTM layer with 64 neurons

model.add(LSTM(64,return_sequences=True))

model.add(LSTM(64))

model.add(Dense(64,activation='relu'))

model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

AttributeError: 'int' object has no attribute 'lower'